# Generador aleatorio de Laberintos

### Algoritmo de busqueda aleatoria en profundidad

Considere una cuadricula de celdas en la que cada celda comienza con cuatro paredes. A partir de una celda aleatoria, la computadora selecciona una celda vecina aleatoria que aún no ha sido visitada. La computadora elimina la pared entre las dos celdas y marca la nueva celda como visitada, y la agrega a la pila para facilitar el seguimiento. La computadora continúa este proceso, y una celda que no tiene vecinos no visitados se considera un callejón sin salida. Cuando se encuentra en un callejón sin salida, retrocede por el camino hasta que llega a una celda con un vecino no visitado, continuando la generación del camino al visitar esta nueva celda no visitada (creando un nuevo cruce). Este proceso continúa hasta que se han visitado todas las celdas, lo que hace que la computadora retroceda hasta la celda inicial. Podemos estar seguros de que cada celda es visitada. 

### Implementación iterativa

    1. Elija la celda inicial, marque la visita y presione la pila
    2. Mientras que la pila no está vacía
        1. Abre una celda de la pila y haz que sea una célula actual
        2. Si la celda actual tiene vecinos que no han sido visitados
            1. Empuja la celda actual a la pila
            2. Elige a uno de los vecinos no visitados
            3. Quitar la pared entre la celda actual y la celda elegida
            4. Marcar la celda elegida como visitada y empujarla a la pila

#### Tamaños de laberintos

1 -> 7 x 7
<br>
2 -> 10 x 10
<br>
3 -> 14 x 14
<br>
4 -> 20 x 20
<br>
5 -> 25 x 25
<br>
6 -> 28 x 28
<br>
7 -> 35 x 35
<br>
8 -> 48 x 48
<br>
9 -> 70 x 70

#### Ejemplo de la generación de un laberinto

<img src="https://imagetolink.com/ib/oVfyj9jaYv.png" width=500px height=500px alt="laberinto"/>

#### Ejemplo de la busqueda del laberinto

<img src="https://imagetolink.com/ib/qv9FaFvM3b.png" width=500px height=500px>

### Codigo

Importación de las librerias

In [3]:
import pygame
import sys
import random
import os
from collections import deque

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


<b>Implementacion del algoritmo<b>

In [4]:
class Casilla:
    def __init__(self, game, x, y, TX, TY):
        self.game = game
        
        self.x = x
        self.y = y
        self.TX = TX
        self.TY = TY
        
        self.corners = [
            (x * TX, y * TY),
            (x * TX + TX, y * TY),
            (x * TX + TX, y * TY + TY),
            (x * TX, y * TY + TY),
            (x * TX, y * TY)
        ]
        
        self.walls = [True, True, True, True]
        self.visited = False
        
    def render(self):
        if self.visited:
            visitedColor = pygame.draw.rect(self.game.pantalla, self.game.VisitCOLOR, 
                (self.x * self.TX, self.y * self.TY, self.TX, self.TY))
            
        for i in range(len(self.walls)):
            if self.walls[i]:
                self.drawBorderLine(self.corners[i][0], self.corners[i][1],
                    self.corners[i + 1][0], self.corners[i + 1][1])
                
    def marcas(self):
        if self.x == 0 and self.y == 0:
            marcaInicio = pygame.draw.rect(self.game.pantalla, self.game.endCOLOR, 
                (self.x * self.TX, self.y * self.TY, self.TX - 1, self.TY - 1))
        
        if self.x == self.game.COLUMNAS - 1 and self.y == self.game.FILAS - 1:
            marcaFinal = pygame.draw.rect(self.game.pantalla, self.game.endCOLOR, 
                ((self.x * self.TX) + 2, (self.y * self.TY) + 2, self.TX, self.TY))
                
    def drawBorderLine(self, x1, y1, x2, y2):
        borderLine = pygame.draw.line(self.game.pantalla, self.game.WallCOLOR,
            (x1, y1), (x2, y2))
        
    def check_vecinos(self):
        x = self.x
        y = self.y
        
        posicionVecinos = [
            (x, y + 1),
            (x, y - 1),
            (x + 1, y),
            (x - 1, y)
        ]
        
        vecinos = []
        
        for i in range(len(posicionVecinos)):
            checkIndice = self.indice(posicionVecinos[i][0], posicionVecinos[i][1])
            
            if checkIndice is not None:
                vecinoEnCheckeo = self.game.casillas[checkIndice]
                
                if not vecinoEnCheckeo.visited:
                    vecinos.append(vecinoEnCheckeo)
                    
        if len(vecinos) > 0:
            num_rnd = random.randrange(len(vecinos))
            return vecinos[num_rnd]
        
        return None
    
    def indice(self, x, y):
        if x < 0 or x > self.game.COLUMNAS - 1 or y < 0 or y > self.game.FILAS - 1:
            return None
        
        return y * self.game.COLUMNAS + x
    
    def cursor(self):
        drawCursor = pygame.draw.rect(self.game.pantalla, self.game.CursorCOLOR,
            (self.x * self.TX, self.y * self.TY, self.TX, self.TY))
        
        
class Game:
    #colores
    def __init__(self):
        pygame.init()
        
        self.BackgraundCOLOR = (232, 232, 232)
        self.WallCOLOR = (30, 30, 30)
        self.CursorCOLOR = (20, 80, 220)
        self.VisitCOLOR = (160, 150, 230)
        self.BfsColor = (220, 220, 220)
        self.PathCOLOR = (16, 170, 252)
        self.TextCOLOR = (28, 28, 28)
        self.endCOLOR = (230, 120, 120)
        
        self.TX = 50
        self.TY = 50
        
        self.RESOLUCION = (700, 700)
        self.FPS = 60
        
        self.stack = []
        self.casillas = []
        
        self.programaEjecutandose = True
        self.comenzar = False
        self.end = False
        self.elegirTamano = True
        
        self.pantalla = pygame.display.set_mode(self.RESOLUCION)
        self.reloj = pygame.time.Clock()
        
        self.nombreArchivo = 'laberinto.png'
        self.bfs_path = None
            
        self.bfsActivo = True
        
        self.show_text = False
        self.text_timer = 0

    def inicializarDimensiones(self):
        self.FILAS = int(self.RESOLUCION[1] / self.TY)
        self.COLUMNAS = int(self.RESOLUCION[0] / self.TX)
        
    def arrayObjetosCasilla(self):
        for i in range(self.FILAS):
            for j in range(self.COLUMNAS):
                casilla = Casilla(self, j, i, self.TX, self.TY)
                self.casillas.append(casilla)
    
    def inicializador(self):
        self.actual = self.casillas[0]
        self.actual.visitada = True
        self.stack.append(self.actual)
        
    def abrirCamino(self, actual, elegida):
        lr = actual.x - elegida.x
        
        if lr == -1:
            actual.walls[1] = False
            elegida.walls[3] = False
        elif lr == 1:
            actual.walls[3] = False
            elegida.walls[1] = False
            
        tb = actual.y - elegida.y
        
        if tb == -1:
            actual.walls[2] = False
            elegida.walls[0] = False
        elif tb == 1:
            actual.walls[0] = False
            elegida.walls[2] = False
    
    def reinicialPrograma(self):
        self.stack.clear()
        self.casillas.clear()
        self.end = False
        self.bfs_path = None
        self.arrayObjetosCasilla()
        self.inicializador()
    
    def printTexto(self, surface, texto, size, x, y, qcolor):
        font = pygame.font.SysFont('serif', size)
        text_surface = font.render(texto, True, qcolor)
        text_rect = text_surface.get_rect()
        text_rect.center = (x, y)
        surface.blit(text_surface, text_rect)
        
    def texto(self):
        centroX = int(self.RESOLUCION[0] / 2)
        centroY = int(self.RESOLUCION[1] / 2)
        
        if self.elegirTamano:
            self.printTexto(self.pantalla, 'Selecciona el tamaño del laberinto (1-9)', 
                            int(self.RESOLUCION[0] / 25), centroX, centroY - 100, self.TextCOLOR)
            self.printTexto(self.pantalla, 'Para guardar el laberinto presiona "S" al finalizar',
                            int(self.RESOLUCION[0] / 30), centroX, centroY - 50, self.TextCOLOR),
            self.printTexto(self.pantalla, 'Para mostrar el camino mas corto presione "A" al finalizar',
                            int(self.RESOLUCION[0] / 30), centroX, centroY, self.TextCOLOR)
        elif not self.comenzar:
            self.printTexto(self.pantalla, 'Pulsa ENTER para comenzar', 
                            int(self.RESOLUCION[0] / 20), centroX, centroY, self.TextCOLOR)
        if self.show_text:
            self.printTexto(self.pantalla, 'Imagen guardada!', 
                            int(self.RESOLUCION[0] / 25), centroX, centroY + 100, self.BackgraundCOLOR)
            
            
    def dibujarLaberinto(self):
        for i in range(len(self.casillas)):
            self.casillas[i].render()
            self.casillas[i].marcas()
        
        if self.bfsActivo and self.bfs_path:
            for (x, y) in self.bfs_path:
                pygame.draw.rect(self.pantalla, self.BfsColor, 
                                 (x * self.TX + self.TX // 5, y * self.TY + self.TY // 5, self.TX // 3, self.TY // 3))

    def laberintoGenerandose(self):
        if len(self.stack) > 0:
            self.dibujarLaberinto()
            
            self.actual = self.stack.pop()
            elegida = self.actual.check_vecinos()
            
            if elegida:
                elegida.cursor()
                self.stack.append(self.actual)
                self.abrirCamino(self.actual, elegida)
                elegida.visited = True
                self.stack.append(elegida)
            else:
                self.actual.cursor()
        else:
            self.dibujarLaberinto()
            self.end = True

    def bfs(self):
        start = (0, 0)
        goal = (self.FILAS - 1, self.COLUMNAS - 1)
        queue = deque([start])
        came_from = {start: None}

        while queue:
            current = queue.popleft()
            if current == goal:
                break

            x, y = current
            neighbors = self.obtenerVecinos(x, y)
            for neighbor in neighbors:
                if neighbor not in came_from:
                    queue.append(neighbor)
                    came_from[neighbor] = current

        self.bfs_path = self.obtenerCamino(came_from, start, goal)

    def obtenerVecinos(self, x, y):
        neighbors = []
        index = y * self.COLUMNAS + x
        cell = self.casillas[index]

        if not cell.walls[0] and y > 0:  # up
            neighbors.append((x, y - 1))
        if not cell.walls[1] and x < self.COLUMNAS - 1:
            neighbors.append((x + 1, y))
        if not cell.walls[2] and y < self.FILAS - 1:
            neighbors.append((x, y + 1))
        if not cell.walls[3] and x > 0:
            neighbors.append((x - 1, y))

        return neighbors

    def obtenerCamino(self, came_from, start, goal):
        path = []
        current = goal
        while current != start:
            path.append(current)
            current = came_from[current]
        path.append(start)
        path.reverse()
        return path

    def actualizacion(self):
        pygame.display.set_caption(str(int(self.reloj.get_fps())))
        self.pantalla.fill(self.BackgraundCOLOR)
        
        if self.comenzar:
            self.laberintoGenerandose()
        
        self.texto()
        
        if self.show_text and pygame.time.get_ticks() - self.text_timer > 100:
            self.show_text = False
        
        pygame.display.flip()
        self.reloj.tick(self.FPS)
        
    def checkEvent(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        
            if event.type == pygame.KEYDOWN:
                if self.elegirTamano:
                    if event.key == pygame.K_1:
                        self.TX, self.TY = 100, 100
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_2:
                        self.TX, self.TY = 70, 70
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_3:
                        self.TX, self.TY = 50, 50
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_4:
                        self.TX, self.TY = 35, 35
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_5:
                        self.TX, self.TY = 28, 28
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_6:
                        self.TX, self.TY = 25, 25
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_7:
                        self.TX, self.TY = 20, 20
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_8:
                        self.TX, self.TY = 14, 14
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    elif event.key == pygame.K_9:
                        self.TX, self.TY = 10, 10
                        self.elegirTamano = False
                        self.inicializarDimensiones()
                        self.reinicialPrograma()
                    
                
                elif event.key == pygame.K_RETURN:
                    if not self.comenzar:
                        self.comenzar = True
                    
                elif event.key == pygame.K_SPACE:
                    if self.comenzar:
                        self.reinicialPrograma()
                        
                elif event.key == pygame.K_r:
                    self.elegirTamano = True
                    self.comenzar = False
                    self.end = False
                    self.stack.clear()
                    self.casillas.clear()
                    
                elif event.key == pygame.K_s:
                    if self.end:
                        self.guardarImagen()
                         
                elif event.key == pygame.K_a:
                    if self.end:
                        self.bfsActivo = not self.bfsActivo
                        if self.bfsActivo:
                            self.bfs()
                    
                elif event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    sys.exit()
                    
    def numeroArchivo(self):
        if not os.path.exists(self.nombreArchivo):
            with open(self.nombreArchivo, 'w') as f:
                f.write('0')
                
        with open(self.nombreArchivo, 'r') as f:
            contador = int(f.read().strip())

        contador += 1

        with open(self.nombreArchivo, 'w') as f:
            f.write(str(contador))
    
        return f"laberintos/laberinto{contador}.png"

    
    def guardarImagen(self):
        archivo = self.numeroArchivo()
        pygame.image.save(self.pantalla, archivo)
        self.show_text = True
        self.text_timer = pygame.time.get_ticks()

    def run(self):
        while self.programaEjecutandose:
            self.checkEvent()
            self.actualizacion()
            
if __name__ == '__main__':
    game = Game()
    game.run()           

SystemExit: 

D:\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
